In [1]:
import numpy as np

In [2]:
# counterfactual regret minimization (CFR) for solving Kuhn Poker
PLAYER_0 = 0
PLAYER_1 = 1
# player 0 always starts and owns the first card in cards = DECK.dopy()
DECK = np.array([1, 2, 3])
# actions are 'pass' (with action_id 0) and 'bet' (with action_id 1)
ACTIONS = ['p', 'b']


def terminal_utility(cards, history, player):
    if len(history) < 2:
        # history not terminal
        return None

    if cards[player] > cards[1 - player]:
        higher_card_multiplier = 1
    else:
        higher_card_multiplier = -1
    if player == PLAYER_0:
        player_multiplier = 1
    else:
        player_multiplier = -1

    if history == "pp":
        return higher_card_multiplier * 1.0
    elif history == "pbp":
        return player_multiplier * -1.0
    elif history == "pbb":
        return higher_card_multiplier * 2.0
    elif history == "bp":
        return player_multiplier * 1.0
    elif history == "bb":
        return higher_card_multiplier * 2.0

In [3]:
class InfoSet:
    def __init__(self): 
        self.regret_sum = np.array([0.0, 0.0])
        self.strategy_sum = np.array([0.0, 0.0])

    def strategy(self):
        regret = self.regret_sum.copy()
        regret[regret < 0.0] = 0.0
        normalizing = regret.sum()
        if normalizing > 0.0:
            return regret / normalizing
        else:
            return [0.5, 0.5]

    def avg_strategy(self):
        normalizing = self.strategy_sum.sum()
        if normalizing > 0.0:
            strategy = self.strategy_sum / normalizing
            # clip small probabilities
            strategy[strategy < 1e-4] = 0.0
            return strategy / strategy.sum()
        else:
            return [0.5, 0.5]

    def __repr__(self):
        return str(self.avg_strategy())


def cfr(info_sets, cards, history, prob_0, prob_1):
    # prob_0 and prob_1 are probabilities that players 0 and 1 play to the
    # current information set respectively
    current_player = len(history) % 2
    u = terminal_utility(cards, history, current_player)
    if u is not None:
        return u

    info_set_key = "card: {:d} - history: {}".format(cards[current_player], history)
    info_set = info_sets.get(info_set_key, InfoSet())

    expected_node_value = 0.0
    action_values = np.array([0.0, 0.0])
    strategy = info_set.strategy()

    for i in range(len(ACTIONS)):
        if current_player == PLAYER_0:
            action_values[i] = -cfr(
                info_sets,
                cards,
                history + ACTIONS[i],
                prob_0 * strategy[i],
                prob_1,
            )
        else:
            action_values[i] = -cfr(
                info_sets,
                cards,
                history + ACTIONS[i],
                prob_0,
                prob_1 * strategy[i],
            )
        expected_node_value += strategy[i] * action_values[i]

    # update the regrets
    if current_player == PLAYER_0:
        info_set.regret_sum += np.multiply(action_values - expected_node_value, prob_1)
        info_set.strategy_sum += np.multiply(strategy, prob_0)
    else:
        info_set.regret_sum += np.multiply(action_values - expected_node_value, prob_0)
        info_set.strategy_sum += np.multiply(strategy, prob_1)

    info_sets[info_set_key] = info_set
    return expected_node_value

In [4]:
# solve the game
NUM_ITER = 2000000

info_sets = {}
util = 0.0
for _ in range(NUM_ITER):
    cards = DECK.copy()
    np.random.shuffle(cards)
    util += cfr(info_sets, cards, "", 1.0, 1.0)

util / NUM_ITER

-0.05656352745539217

In [5]:
policy = {k: v.avg_strategy() for k, v in info_sets.items()}
policy

{'card: 3 - history: pb': array([0., 1.]),
 'card: 2 - history: p': array([1., 0.]),
 'card: 2 - history: b': array([0.66634135, 0.33365865]),
 'card: 3 - history: ': array([0.25711416, 0.74288584]),
 'card: 1 - history: pb': array([1., 0.]),
 'card: 3 - history: p': array([0., 1.]),
 'card: 3 - history: b': array([0., 1.]),
 'card: 1 - history: ': array([0.75391227, 0.24608773]),
 'card: 2 - history: pb': array([0.41828635, 0.58171365]),
 'card: 1 - history: p': array([0.66314706, 0.33685294]),
 'card: 1 - history: b': array([1., 0.]),
 'card: 2 - history: ': array([1., 0.])}

In [6]:
# calculate the expected value for the first player given the obtained policy (same as above)
NUM_GAMES = 500000


def random_game_value():
    cards = DECK.copy()
    np.random.shuffle(cards)
    history = ""
    current_player = PLAYER_0

    while terminal_utility(cards, history, current_player) is None:
        info_set_key = "card: {:d} - history: {}".format(cards[current_player], history)
        action = np.random.choice([0, 1], p=policy[info_set_key])
        history += ACTIONS[action]
        current_player = 1 - current_player

    return terminal_utility(cards, history, PLAYER_0)

value = 0.0
for _ in range(NUM_GAMES):
    value += random_game_value()

value / NUM_GAMES

-0.05681

In [7]:
# expected value for the first player based on https://en.wikipedia.org/wiki/Kuhn_poker
-1/18

-0.05555555555555555